In [1]:
import os
os.chdir('/mnt/c/Users/ralvin/OneDrive - Reliant Health Partners/Documents/RHP_dev_RA/Automate_skyvia')
import pandas as pd
from simple_salesforce import Salesforce, SalesforceLogin
import pyodbc, sys, time 
import requests
import numpy as np
from datetime import datetime
from helper_functions_v2 import create_df, updated_data_pull, upsert, delete_record, insert_records, salesforce_connection, read_sftp_data
from openai import OpenAI
import ast

pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 

sf = salesforce_connection(sandbox=True)
sf_prod = salesforce_connection(sandbox=False)


Connected to Salesforce sandbox
Connected to Salesforce Prod


In [2]:
provider_data_test = updated_data_pull(['Provider_TIN__c'], sf)
provider_data_real = updated_data_pull(['Provider_TIN__c'], sf_prod)

pulled Provider_TIN__c
pulled Provider_TIN__c


In [5]:
temp = provider_data_real.loc[:, ['Special_Repricing__c', 'Notes__c']]
temp[temp['Special_Repricing__c']!=False].drop_duplicates(subset=['Notes__c']).head(10)

list_of_notes = list(temp[temp['Special_Repricing__c']!=False].drop_duplicates(subset=['Notes__c'])['Notes__c'])


client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

def LLM(prompt):


  completion = client.chat.completions.create(
  model="meta/llama-3.3-70b-instruct",
  messages=[{"role":"user","content": prompt}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

  output = ""
  for chunk in completion:
    if chunk.choices[0].delta.content is not None:
      # print(chunk.choices[0].delta.content, end="")
      output += chunk.choices[0].delta.content


  return output





def parse_output(output):
    cluster_labels = []
    cluster_definitions = {}

    lines = output.split("\n")
    cluster_section_started = False
    cluster_labels_temp = []

    for line in lines:
        line = line.strip()

        # Identify and parse cluster definitions
        if line.startswith("* Cluster "):
            parts = line.split(":")
            if len(parts) >= 2:
                try:
                    cluster_id = int(parts[0].split(" ")[-1])
                    definition = ":".join(parts[1:]).strip()
                    cluster_definitions[cluster_id] = definition
                except ValueError:
                    continue  # Skip invalid cluster lines

        # Identify the start of the cluster labels list
        if line.startswith("The cluster labels list is:"):
            cluster_section_started = True
            continue

        # Collect cluster labels list lines
        if cluster_section_started:
            if line.startswith("[") or line.endswith("]") or "," in line:
                cluster_labels_temp.append(line.strip(","))
            elif line.startswith("```"):
                # End of cluster labels section
                cluster_section_started = False

    # Join and evaluate the complete cluster labels list
    if cluster_labels_temp:
        try:
            cluster_labels = eval("".join(cluster_labels_temp))
        except Exception as e:
            print(f"Error parsing cluster labels: {e}")


    cluster_label_string = lines[-1]
    fixed_string = cluster_label_string.rstrip(", ") + "]"
    converted_list = ast.literal_eval(fixed_string)

    return {"cluster_labels":converted_list , "definitions": cluster_definitions}


In [10]:
prompt = f"""
Given a list of strings containing notes on a specific insurance provider,
analyze the notes and group them into categories based on their meanings.
Do not define specific categories upfront; create them based on the content
of the notes. The overall goal is to standardize the strings;
Once a class has been determined, store that value into a list for each string in the input list.
This here is the list of stings: {list_of_notes[:20]}. Only generate a list containing the cluster_labels and another list for the cluster_definitions. 
"""
output = LLM(prompt)

# response = parse_output(output)
# response

In [7]:
list_of_notes[:100]

['8/23/24 - bk - Non Par for QON/Broadspire',
 'appealed pmt - 100% went to deduct',
 'process 100%',
 '6/18/24 - bk - Please red flag for Novanet.',
 'appeals CT work for FS after 4/1/2015',
 'appeals CT work comp',
 'appealed wc',
 'appeals CT work comp below FS',
 'provider balance bills, process 100%',
 'process in full',
 'provider does not accept discount, PROCESS at 100%',
 '12/22/24 - bk - Non par for Paradigm',
 '*Medben* - update rate for 86015',
 '11/19/24 - bk - Non par TIN.',
 '7/2/24 - bk - please red flag for Novanet.\r\n\r\nappeals MN wc, under 100 beds',
 '10/3/24 - bk - Non Par for Novanet',
 'standard negot discount of 10%',
 'Feder pays upfront - process at 100%',
 'member pays upfront',
 'they want rates closer to "BCBS of Alabama" which averages a 20% discount. So dont discount more than 20%',
 'member pays upfront or bb',
 'Provider disagrees with discount, will bb',
 'appeal large MO WC to State, try 30% on primary and 50% on secondary,',
 'appeal SD work comp f

In [11]:
print(output)



### Insurance Notes Clustering

To cluster the insurance notes, we'll use the K-Means clustering algorithm from scikit-learn. We'll first preprocess the notes by converting them into numerical vectors using the TfidfVectorizer.

```python
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# List of insurance notes
notes = [
    '8/23/24 - bk - Non Par for QON/Broadspire',
    'appealed pmt - 100% went to deduct',
    'process 100%',
    '6/18/24 - bk - Please red flag for Novanet.',
    'appeals CT work for FS after 4/1/2015',
    'appeals CT work comp',
    'appealed wc',
    'appeals CT work comp below FS',
    'provider balance bills, process 100%',
    'process in full',
    'provider does not accept discount, PROCESS at 100%',
    '12/22/24 - bk - Non par for Paradigm',
    '*Medben* - update rate for 86015',
    '11/19/24 - bk - Non par TIN.',
    '7/2/24 - bk - please red

In [80]:
len(response['cluster_labels'])

19